# Résolution problème Hock-Schittkowski 47

In [ ]:
using BenchmarkTools, ForwardDiff, Distributions

In [ ]:
include("../src/enlsip_functions_castor2.jl")

In [ ]:
n = 5
m = 4
nb_eq = 3
nb_constraints = 3

In [ ]:
# Residuals

function r(x::Vector)
    return [x[1]-x[2], x[2]-x[3], (x[3]-x[4])^2 ,(x[4]-x[5])^2]
end

resHS47 = ResidualsEval(0)

function (resHS47::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)

    # Evaluate the residuals
    if abs(resHS47.ctrl) == 1
        rx[:] = r(x)

    # The jacobian is computed analytically
    elseif resHS47.ctrl == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [ ]:
function c(x::Vector)
    return [x[1]+x[2]^2+x[3]^2-3, x[2]-x[3]^2+x[4]-1, x[1]*x[5]-1]
end
    
consHS47 = ConstraintsEval(0)

function (consHS47::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)

    # Evaluate the constraints
    if abs(consHS47.ctrl) == 1
        cx[:] = c(x)
    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif consHS47.ctrl == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end

In [ ]:
x0 = [2, √2, -1, 2-√2, 0.5]
enlsipHS47 = enlsip(x0,resHS47,consHS47,n,m,nb_eq,nb_constraints)

# Version étendue
    
$$\left\{  
\begin{array}{lll} 
\min_x F(x) \\ 
\text{s.c.}\\
 c_k(x)=0 \quad \text{ pour } 1\leq k \leq 3(n-1)/4
\end{array} \right.$$

$$F(x)=\sum_{i=1}^{(n-1)/4} \left[ (x_{j+1}-x_{j+2})^2 +(x_{j+2}-x_{j+3})^2 + (x_{j+3}-x_{j+4})^4+ (x_{j+4}-x_{j+5})^4 \right]$$

avec $j=4(i-1)$.

$$\begin{array}{lll}
& c_k(x) = x_{l+1} + x_{l+2}^2+x_{l+3}^2 -3, &\mod(k,3)=1 \\
& c_k(x) = x_{l+2} + x_{l+3}^2+x_{l+4} -1, &\mod(k,3)=2  \\
& c_k(x) = x_{l+1} x_{l+5}-1, &\mod(k,3)=0 
\end{array}
$$

avec $l=4\left\lfloor\dfrac{k-1}{3}\right\rfloor$

In [ ]:
n  = 9 #997 # must be > 5 and mod(n,4) = 1
N = div(n-1,4)
m = 4N
nb_eq = 3*N
nb_constraints = nb_eq

In [ ]:
# Residuals

function r(x::Vector)
    n = length(x)
    N = div(n-1,4)
    
    rx1 = [x[4i-3] - x[4i-2] for i=1:N]
    rx2 = [x[4i-2] - x[4i-1] for i=1:N]
    rx3 = [(x[4i-1] - x[4i])^2 for i=1:N]
    rx4 = [(x[4i] - x[4i+1])^2 for i=1:N]
    return [rx1;rx2;rx3;rx4]
end

resHS47ext = ResidualsEval(0)

function (resHS47ext::ResidualsEval)(x::Vector, rx::Vector, J::Matrix)

    # Evaluate the residuals
    if abs(resHS47ext.ctrl) == 1
        rx[:] = r(x)

    # The jacobian is computed analytically
    elseif resHS47ext.ctrl == 2
        J[:] = ForwardDiff.jacobian(r,x)
    end
    return
end

In [ ]:
function c(x::Vector)
    N = div(n-1,4)
    cx = Vector(undef,3N)
    
    for k = 1:3N
        l = 4*div(k-1,3)
        if mod(k,3) == 1
            cx[k] = x[l+1] + x[l+2]^2 + x[l+3]^2 - 3
        elseif k % 3 == 2
            cx[k] = x[l+2] + x[l+3] + x[l+4] - 1
        else
            cx[k] = x[l+1] * x[l+5] - 1
        end
    end
    return cx
end
    
consHS47ext = ConstraintsEval(0)

function (consHS47ext::ConstraintsEval)(x::Vector, cx::Vector, A::Matrix)

    # Evaluate the constraints
    if abs(consHS47ext.ctrl) == 1
        cx[:] = c(x)
    # The jacobian is computed numerically if ctrl is set to 0 on return
    elseif consHS47ext.ctrl == 2
        A[:] = ForwardDiff.jacobian(c,x)
    end
    return
end

In [ ]:
x0_small = [2.0, 1.5, -1.0, 0.5]
x0 = [x0_small[mod(i-1,4)+1] for i=1:n]

enlsipHS47ext = enlsip(x0,resHS47ext,consHS47ext,n,m,nb_eq,nb_constraints,ε_c=1e-8)

In [ ]:
@printf "Valeur ENLSIP-Julia : %.10e\n\n" enlsipHS47ext.obj_value

# Résolution avec Ipopt

In [ ]:
using JuMP, Ipopt

In [ ]:
n  = 997 # must be > 5 and mod(n,4) = 1
N = div(n-1,4)
m = 4N
nb_eq = 3*N
nb_constraints = nb_eq

x0_small = [2.0, 1.5, -1.0, 0.5]
x0 = [x0_small[mod(i-1,4)+1] for i=1:n]

# Starting point

model = Model(Ipopt.Optimizer)
@variable(model,x[i=1:n],start=x0_small[mod(i-1,4)+1])

for k=1:3N
    l = 4*div(k-1,3)
    if mod(k,3) == 1
        @NLconstraint(model, x[l+1] + x[l+2]^2 + x[l+3]^2 == 3)
    elseif k % 3 == 2
        @NLconstraint(model, x[l+2] + x[l+3] + x[l+4] == 1)
    else
        @NLconstraint(model, x[l+1] * x[l+5] == 1)
    end
end

@NLobjective(model, Min, sum((x[4i-3] - x[4i-2])^2 + (x[4i-2] - x[4i-1])^2 + 
        (x[4i-1] - x[4i])^4 + (x[4i] - x[4i+1])^4 for i=1:N))

JuMP.optimize!(model)

In [ ]:
@printf "Valeur ENLSIP-Julia : %.10e\n\n" enlsipHS47ext.obj_value
@printf "Valeur Ipopt : %.10e\n\n" objective_value(model)